In [151]:
import numpy as np
    
#Example hexadecimal string inputs
ex1 = 'd2fe28'
ex2 = '38006F45291200'
ex3 = 'ee00d40c823060'
ex4 = '8A004A801A8002F478'
ex5 = '620080001611562C8802118E34'
ex6 = 'C0015000016115A2E0802F182340'
ex7 = 'A0016C880162017C3686B18A3D4780'
#The actual puzzle input:
EX = 'E0525D9802FA00B80021B13E2D4260004321DC648D729DD67B2412009966D76C0159ED274F6921402E9FD4AC1B0F652CD339D7B82240083C9A54E819802B369DC0082CF90CF9280081727DAF41E6A5C1B9B8E41A4F31A4EF67E2009834015986F9ABE41E7D6080213931CB004270DE5DD4C010E00D50401B8A708E3F80021F0BE0A43D9E460007E62ACEE7F9FB4491BC2260090A573A876B1BC4D679BA7A642401434937C911CD984910490CCFC27CC7EE686009CFC57EC0149CEFE4D135A0C200C0F401298BCF265377F79C279F540279ACCE5A820CB044B62299291C0198025401AA00021D1822BC5C100763A4698FB350E6184C00A9820200FAF00244998F67D59998F67D5A93ECB0D6E0164D709A47F5AEB6612D1B1AC788846008780252555097F51F263A1CA00C4D0946B92669EE47315060081206C96208B0B2610E7B389737F3E2006D66C1A1D4ABEC3E1003A3B0805D337C2F4FA5CD83CE7DA67A304E9BEEF32DCEF08A400020B1967FC2660084BC77BAC3F847B004E6CA26CA140095003900BAA3002140087003D40080022E8C00870039400E1002D400F10038C00D100218038F400B6100229500226699FEB9F9B098021A00800021507627C321006E24C5784B160C014A0054A64E64BB5459DE821803324093AEB3254600B4BF75C50D0046562F72B1793004667B6E78EFC0139FD534733409232D7742E402850803F1FA3143D00042226C4A8B800084C528FD1527E98D5EB45C6003FE7F7FCBA000A1E600FC5A8311F08010983F0BA0890021F1B61CC4620140EC010100762DC4C8720008641E89F0866259AF460C015D00564F71ED2935993A539C0F9AA6B0786008D80233514594F43CDD31F585005A25C3430047401194EA649E87E0CA801D320D2971C95CAA380393AF131F94F9E0499A775460'

#function which converts a decimal to binary, formatted with n bits
def getbin(x, n=0):
    return format(x, 'b').zfill(n)

#We give this function a whole bitstring which we know to begin with a literal
#It will whittle it down 5 bits at a time 
#Saves the parsed number and returns the remaining bitstring
#Inputs: strng: the bitstring 
def literalparser(strng):    
        
    #hold the bit strings which represent the literals
    subliterals = []
    
    #pick the first five bits
    currentfive = strng[0:5]
    
    #while the first bit of the surrent five bits is 1, continue parsing
    while currentfive[0] == '1':
        #store the binary number:
        subliterals.append(currentfive[1:5])
        
        #shorten the string
        strng = strng[5:]
        
        #reset the 'current' five bits
        currentfive = strng[0:5]
        
    #the loop has been broken, hence we are on the last five bits, so parse one more time:
    subliterals.append(currentfive[1:5])
    strng = strng[5:]
    
    #merge the subliterals to find the number represented by this literal
    parsednumber = ''.join(subliterals)
    
    #For Part 1 we only care about version numbers so only need to return the remaining string
    return strng#,int(parsednumber, base=2)


#For now we don't need to keep track of subpackets too much,
#So the operator functions just strip a given bitstring of the length information at the start
#And return the remaining string

#Length Type ID = 1
#Input is a bitstring
def operatorparser0(strng):
    
    #find the LENGTH of the subpackets to follow:
    L = int(strng[0:15], base=2)
    #adjust string
    strng = strng[15:]
    
    return strng
    
#Length Type ID = 1
#Input is a bitstring
def operatorparser1(strng):

    #find the COUNT of the subpackets to follow:
    N = int(strng[0:11], base=2)
    #adjust string
    strng = strng[11:]
    
    return strng
       
#This function repeatedly checks the beginning of each (sub)packet for its version number
#Stores it, and then applies the appropriate parser function
#Takes strng:hexadecimal string input
def fullparser(strng):
    
    #convert hexadecimal to binary with correct formatting
    bitstring = getbin(int(strng, base=16),4*len(strng))
    #initialise somehwere to save version numbers
    versionarr = np.array([])
    
    #minimum length of meaningful packet is 11
    while len(bitstring) > 11:
        #check version and typeid, parse appropriately
        version = int(bitstring[0:3], base=2)
        typeid = int(bitstring[3:6], base=2)
        versionarr = np.append(versionarr, version)

        if typeid == 4:
            #This packet is a literal
            bitstring = bitstring[6:]
            bitstring = literalparser(bitstring)
        else:
            #This packet is an operator
            lengthtypeid = bitstring[6]
            bitstring = bitstring[7:]
            if lengthtypeid == '0':
                #Length Type ID 0
                bitstring = operatorparser0(bitstring)
            else:
                #Length Type ID 1
                bitstring = operatorparser1(bitstring)
    
    #return sum of version numbers
    return np.sum(versionarr)

#Part 1
print("The sum of the version numbers of all the packets: " + str(fullparser(EX)))

The sum of the version numbers of all the packets: 940.0
